In [40]:
import pandas as pd
import nltk
from datetime import datetime
from nltk.stem.snowball import SnowballStemmer
import re
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from nltk.stem.snowball import SnowballStemmer
from PIL import Image
import matplotlib.pyplot as plt


In [47]:
data=pd.read_csv("data\\kred_unic_all.csv", encoding='cp1251',delimiter=';')
data['TEXT']

0         клиент в настоящий момент находится на отдыхе ...
1         ет договоров в работе, счет закрывать не будут...
2                                      плохое фин положение
3                   клиент попадает в просрочку. нет залога
4         клиент поставляет оборудование в сеть магазино...
5              кредит уже выдан, новый продукт не интересен
6          выдача кредита 2216/бизнес-оборот в декабре 2016
7         сфера деятельности клиента- арендный бизнес. и...
8                                                22.02.2017
9         достигнута договоренность о встрече на 08.02.2017
10                                       встреча 14 февраля
11        назначена встреча с клиентом на 21 февраля 201...
12                      на территории клиента в 16-00 09/02
13          назначена встреча с лпр на территории у клиента
14        руководитель и гл. бухгалтер отсутствуют. буде...
15        с клиентом проводили встречу в декабре прошлог...
16                                      

In [13]:
import datetime
import re

def valid_date(string):
    string=string.split(' ')
    parsed_string=''
    for token in string:
            try:
                 parsed_string+=' '+(re.sub(r'(\d{2}).(\d{2}).?(\d{0,4})', r'дата\время', token))
            except ValueError:
                 parsed_string+=' '+token
    return parsed_string

def delete_punctuation(s):
    return ' '.join((re.sub(r'[№"\'_/.:?!1234567890()%<>;,+#$&\s+]', u' ', s)).split())

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word not in stopwords])

In [8]:
datestring=['+7 (925) 545 98 86 встреча 18.03.2016г. подъеду в офис',
            '12\\12\\2016', '18:00', 
            '12-12-16', 'звонок на 12:00']
for s in datestring:
    print(valid_date(s))


 +7 (925) 5дата\время встреча дата\времяг. подъеду в офис дата\время дата\время дата\время звонок на дата\время
 +7 (925) 5дата\время встреча дата\времяг. подъеду в офис дата\время дата\время дата\время звонок на дата\время
 +7 (925) 5дата\время встреча дата\времяг. подъеду в офис дата\время дата\время дата\время звонок на дата\время
 +7 (925) 5дата\время встреча дата\времяг. подъеду в офис дата\время дата\время дата\время звонок на дата\время
 +7 (925) 5дата\время встреча дата\времяг. подъеду в офис дата\время дата\время дата\время звонок на дата\время


In [ ]:
tokens=['клиент', 'клиенту', 'клиентом', 'клиенты']
for t in tokens:
    print(stemmer.stem(t))

In [4]:
stopwords=[u'и',u'в',u'во',u'что',u'он',u'на',u'я',u'с',u'со',u'как',u'а',u'то',u'все',u'она',u'так',u'его',u'но',u'да',u'ты',u'к',u'у',u'же',
u'за',u'бы',u'по',u'только',u'ее',u'мне',u'было',u'вот',u'от',u'меня',u'еще', u'о',u'из',u'ему',u'теперь',u'когда',u'даже',u'ну',u'вдруг',
u'ли',u'если',u'уже',u'или',u'ни',u'быть',u'был',u'него',u'до',u'вас',u'нибудь',u'опять',u'уж',u'вам',u'ведь',u'там',u'потом',u'себя',u'ничего',
u'ей',u'может',u'они',u'тут',u'где',u'есть',u'надо',u'ней',u'для',u'мы',u'тебя',u'их',u'чем',u'была',u'сам',u'чтоб',u'без',u'будто',u'чего',u'раз',
u'тоже',u'себе',u'под',u'будет',u'ж',u'тогда',u'кто',u'этот',u'того',u'потому',u'этого',u'какой',u'совсем',u'ним',u'здесь',u'этом',u'один',u'почти',u'мой',
u'тем',u'чтобы', u'rur', u'rub', u'ru', u'руб', u'рублей', u'ндс', u'ooo', u'oao', u'без', u'ао', u'зао', u'г', u'ы', u'пао', u'гуп', u'ук',
u'кпк', u'кх', u'бст', u'хк', u'ук', u'январь', u'февраль', u'март',u'апрель',u'май', u'июнь', u'июль', u'август', u'сентябрь', u'октябрь', u'ноябрь', u'декабрь',
u'июл', u'феврал', u'распоряжение', u'облагается', u'n',  u'шт', u'период', u'г', u'eur', u'маы', u'апрел', u'июн', u'январ', u'иуне',
u'иуле', u'дог', u'рублях', u'начисление', u'экз',  u'sd', u'возврат', u'взимание', u'ед', u'е', u'юр', u'а',u'б',u'в',u'г',u'д',u'е',u'ё',
u'ж',u'з',u'и',u'к',u'л',u'м',u'н',u'о',u'п',u'р',u'с',u'т',u'у',u'ф',u'х',u'ц',u'ч',u'ш',u'щ',u'ъ',u'ы',u'ь',u'э',u'ю',u'я', u'р', u'б', u'н', u'нал', u'сч',
u'запрос', u'ип', u'монеты', u'дс', u'бум', u'лоро', u'заявке', u'нальчик', 'a','b','v','g','d','e','e','zh','z','i','k','l','m','n','o','p','r','s','t','u','f','h',
u'вал', u' подразд', u'приема', u'см', u'usd', u'sek', u'nok',  u'орому', u'орым', u'lc', u'usd', u'ов', u'vo', u'осб', u'сзб', u'visa', u'mastercard', u'тс',
u'ммвб', u'фб', u'дату', u'инн', u'кпп', u'ко', u'пс', u'контр', u'кампания', u'бух', u'ген', u'дир', u'кампании', u'через', u'над', u'ксб', u'скб', u'над',
u'списание', u'согл', u'распоряжению', u'свыше', u'виза', u'дог', u'файл', u'spe', u'spx', u'spdz', u'января',u'февраля', u'марта', u'апреля', u'мая', u'июня', u'июля', u'августа', u'сентября',
u'октября', u'ноября', u'декабря', u'зао', u'ооо', u'оао', u'ао', u'без', u'пао', u'гуп', u'ук', u'кпк', u'кх', u'бст', u'хк']

In [46]:
stemmer=SnowballStemmer("russian")

def collect(string):
    for s in string.split(' '):
        if (len(s)<4) & (s not in collection) & (s not in ['нет', 'не', 'др']):
            collection.append(s)

# collection=[]
# data['TEXT'].apply(lambda x: collect(x))
# stopwords=stopwords+collection
stopwords=set(stopwords)

def stemming(words):
    stemmed_words=[]
    for word in words:
        if word not in stopwords:
            stemmed_words.append(stemmer.stem(word))
    return ' '.join(stemmed_words)

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word not in stopwords])

In [32]:
def preprocessing(data):
    data=data.dropna()
    data['TEXT']=data['TEXT'].apply(lambda x: delete_punctuation(valid_date(x.lower())).split())    
    print(data['TEXT'])
#     data=data.loc[[len(data['TEXT'])<7]]
#     data['TEXT']=data['TEXT'].apply(lambda x: x.split())
    data['TEXT']=data['TEXT'].apply(lambda x: stemming(x))
    return data
    

In [48]:
data=preprocessing(data)
data.to_csv('preprocessing_result.csv', sep=';', encoding='cp1251')
print('DONE')
    

0         [клиент, в, настоящий, момент, находится, на, ...
1         [ет, договоров, в, работе, счет, закрывать, не...
2                                  [плохое, фин, положение]
3             [клиент, попадает, в, просрочку, нет, залога]
4         [клиент, поставляет, оборудование, в, сеть, ма...
5         [кредит, уже, выдан, новый, продукт, не, интер...
6              [выдача, кредита, бизнес-оборот, в, декабре]
7         [сфера, деятельности, клиента-, арендный, бизн...
8                                              [дата\время]
9         [достигнута, договоренность, о, встрече, на, д...
10                                       [встреча, февраля]
11        [назначена, встреча, с, клиентом, на, февраля,...
12        [на, территории, клиента, в, дата\время, дата\...
13        [назначена, встреча, с, лпр, на, территории, у...
14        [руководитель, и, гл, бухгалтер, отсутствуют, ...
15        [с, клиентом, проводили, встречу, в, декабре, ...
16                                      

In [ ]:
from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt
import seaborn as sns

vectorizer=CountVectorizer()
vectors=vectorizer.fit_transform(data['TEXT'].values[:50000])
distortions = []
for i in range(20, 70):
    print('Clustering with k-means (%d clusters) ... \n' %(i), end='', flush=True)
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                random_state=48)
   
    km.fit(vectors)
    distortions.append(km.inertia_)
fig=plt.figure(figsize=(45,30))
plt.plot(range(2, 70), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.tight_layout()
fig.savefig('elbow.png')
plt.show()

Clustering with k-means (20 clusters) ... 
Clustering with k-means (21 clusters) ... 
Clustering with k-means (22 clusters) ... 
Clustering with k-means (23 clusters) ... 
Clustering with k-means (24 clusters) ... 
Clustering with k-means (25 clusters) ... 
Clustering with k-means (26 clusters) ... 
Clustering with k-means (27 clusters) ... 
Clustering with k-means (28 clusters) ... 
Clustering with k-means (29 clusters) ... 
Clustering with k-means (30 clusters) ... 
Clustering with k-means (31 clusters) ... 
Clustering with k-means (32 clusters) ... 
Clustering with k-means (33 clusters) ... 
Clustering with k-means (34 clusters) ... 
Clustering with k-means (35 clusters) ... 
Clustering with k-means (36 clusters) ... 
Clustering with k-means (37 clusters) ... 
Clustering with k-means (38 clusters) ... 
Clustering with k-means (39 clusters) ... 
Clustering with k-means (40 clusters) ... 
Clustering with k-means (41 clusters) ... 
Clustering with k-means (42 clusters) ... 
Clustering 

KeyboardInterrupt: 